In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from scipy.spatial.distance import cosine
from tqdm import tqdm

# 설정
n_qubits = 3
shots = 1024
repeats = 10
simulator = AerSimulator()

# 벡터 변환 함수
def counts_to_vector(counts, n=3):
    total = sum(counts.values())
    vec = np.zeros(2**n)
    for bitstring, count in counts.items():
        idx = int(bitstring, 2)
        vec[idx] = count / total
    return vec

# 회로 A: G-F 실험 구조 반복
def circuit_A():
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i in range(n_qubits):
        qc.ry(np.pi/4, i)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.barrier()
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

# 회로 B: 유사 구조 (RY 각도만 무작위)
def circuit_B():
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i in range(n_qubits):
        qc.ry(np.random.uniform(0, np.pi), i)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.barrier()
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

# 회로 C: 완전 무작위 회로
def circuit_C():
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i in range(n_qubits):
        qc.ry(np.random.uniform(0, np.pi), i)
    if np.random.rand() < 0.5:
        qc.cx(0, 1)
    if np.random.rand() < 0.5:
        qc.cx(1, 2)
    qc.barrier()
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

# 회로 실행 함수
def run_circuit(qc):
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc, shots=shots).result()
    counts = result.get_counts()
    return counts_to_vector(counts)

# 실험 실행
sim_A = [run_circuit(circuit_A()) for _ in tqdm(range(repeats), desc="조건 A (구조 반복)")]
sim_B = [run_circuit(circuit_B()) for _ in tqdm(range(repeats), desc="조건 B (유사 구조)")]
sim_C = [run_circuit(circuit_C()) for _ in tqdm(range(repeats), desc="조건 C (랜덤)")]

# 유사도 비교
def pairwise_similarity(group, ref_group):
    sims = []
    ref = np.mean(ref_group, axis=0)
    for vec in group:
        sim = 1 - cosine(vec, ref)
        sims.append(sim)
    return sims

sim_A_vs_C = pairwise_similarity(sim_A, sim_C)
sim_B_vs_C = pairwise_similarity(sim_B, sim_C)
sim_A_internal = pairwise_similarity(sim_A, sim_A)

# 결과 출력
print("\n🔁 조건 A (구조 반복) vs C 유사도")
print("  평균:", np.mean(sim_A_vs_C))
print("  표준편차:", np.std(sim_A_vs_C))

print("\n🔀 조건 B (무작위 구조) vs C 유사도")
print("  평균:", np.mean(sim_B_vs_C))
print("  표준편차:", np.std(sim_B_vs_C))

print("\n📌 조건 A 반복 내 유사도 (자기 정렬)")
print("  평균:", np.mean(sim_A_internal))
print("  표준편차:", np.std(sim_A_internal))
